
## Import necessary libraries

In [1]:
import torch

# Verifica si CUDA (GPU) está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [2]:
# Python libraries
import datetime as dt
import re
import pickle
from tqdm.notebook import tqdm
import os
import sys
import time
import logging
import random
from collections import defaultdict, Counter

# Data Science modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
plt.style.use('ggplot')


# Import nltk modules and download dataset
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop = set(stopwords.words('english'))

# Import Pytorch modules
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.autograd import Variable
from torch.optim import Adam, AdamW

C:\Users\luisl\AppData\Local\Temp\ipykernel_7120\269888456.py:15: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luisl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luisl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\luisl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
import warnings
warnings.filterwarnings("ignore")

## cargar texto resumido 

In [4]:
import pickle

# Define the path to the pickle file (same as before)
ruta_pickle = 'C:/Users/luisl/OneDrive/Escritorio/TFM_IA3/tfm_ia3_nodata/data/preprocessed/text_resumido_v2_completo.pkl'

# Open the file in binary read mode
with open(ruta_pickle, 'rb') as pickle_file:
  # Use pickle.load to deserialize the data and create a new DataFrame
  loaded_df = pickle.load(pickle_file)

print("¡DataFrame cargado del archivo pickle exitosamente!")

¡DataFrame cargado del archivo pickle exitosamente!


In [5]:
loaded_df.head()

,type,date,title,speaker,word_count,decision,rate,next_meeting,next_decision,next_rate,text,summary
0,statement,1994-02-04,FOMC Statement,Alan Greenspan,99,1,3.25,1994-02-28,0,3.25,Chairman Alan Greenspan announced today that t...,I think it's better to take them up later in t...
1,statement,1994-03-22,FOMC Statement,Alan Greenspan,40,1,3.50,1994-04-18,1,3.75,Chairman Alan Greenspan announced today that t...,I am distressed about what happened after the ...
2,statement,1994-04-18,FOMC Statement,Alan Greenspan,37,1,3.75,1994-05-17,1,4.25,Chairman Alan Greenspan announced today that t...,I thought we might change the order [of the ag...
3,statement,1994-05-17,FOMC Statement,Alan Greenspan,57,1,4.25,1994-07-06,0,4.25,"In taking the discount action, the Board appro...",Everybody in this room has a significantly hig...
4,statement,1994-08-16,FOMC Statement,Alan Greenspan,51,1,4.75,1994-09-27,0,4.75,"In taking the discount rate action, the Board ...",I don't know that we've been that precise abou...


In [6]:
train_sent_df = loaded_df

In [7]:
import pandas as pd

# Porcentaje deseado de las filas (e.g., 10%)
porcentaje = 100

# Calcula el número de filas a seleccionar basado en el porcentaje
num_filas = int(len(train_sent_df) * (porcentaje / 100))

# Selecciona aleatoriamente el porcentaje deseado de filas del DataFrame
submuestra_df = train_sent_df.sample(n=num_filas)

submuestra_df = submuestra_df.dropna(subset=['next_decision'])

# Convierte la columna deseada a una lista
X_submuestra = submuestra_df['summary'].to_list()



carga del primer modelo reentrenado

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Define labels
labels = ['negative', 'neutral', 'positive']

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# Define the path to the saved model (assuming the model is saved in a folder)
model_path = "C:/Users/luisl/OneDrive/Escritorio/TFM_IA3/tfm_ia3_nodata/models/best_model_5e-05_64_V2.pt"  # Replace with your actual path

# Function to preprocess text
def preprocess_text(text, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs.input_ids.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    attention_mask = inputs.attention_mask.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return input_ids, attention_mask

# Load the model
model_finbert = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model_finbert.load_state_dict(torch.load(model_path))
model_finbert.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Move to GPU if available

with torch.no_grad():
    model_finbert.eval()

    # Example usage
    text = "The stock market is booming!"
    input_ids, attention_mask = preprocess_text(text, tokenizer)

    # Get predictions
    outputs = model_finbert(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

    # Get the predicted sentiment
    predicted_sentiment = labels[predicted_label]

    print(f"Predicted sentiment for '{text}': {predicted_sentiment}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Predicted sentiment for 'The stock market is booming!': positive


twitter-roberta-base-sentiment Model¶


In [9]:
import torch
import numpy as np
import scipy.special
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# Cargar el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model_roberta = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model_roberta.load_state_dict(torch.load(model_path))
model_roberta.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Mover el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_roberta.to(device)

tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
labels = ['negative', 'neutral', 'positive']

# Asumiendo que X_submuestra es tu lista de textos a analizar
preds_roberta = []
preds_proba_roberta = []

for x in tqdm(X_submuestra, desc="Procesando predicciones"):
    with torch.no_grad():
        # Tokenizar y mover el texto completo a la GPU
        input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs).to(device)
        # Obtener las logits del modelo
        logits = model_roberta(**input_sequence).logits
        # Aplicar softmax en la GPU
        scores = torch.softmax(logits, dim=1).cpu().numpy().squeeze()
    
    sentimiento_general = labels[np.argmax(scores)]
    probabilidad_general = np.max(scores)
    
    preds_roberta.append(sentimiento_general)
    preds_proba_roberta.append(probabilidad_general)


Procesando predicciones: 100%|██████████| 4597/4597 [02:27<00:00, 31.26it/s]


In [10]:
submuestra_df['predicted_roberta'] = preds_roberta; submuestra_df

,type,date,title,speaker,word_count,decision,rate,next_meeting,next_decision,next_rate,text,summary,predicted_roberta
714,presconf_script,2013-03-20,FOMC Press Conference Transcript,Ben Bernanke,391,0,0.00,2013-05-01,0,0.00,"Well, I still believe the following, which is ...","Bernanke: ""Monetary policy is a very blunt ins...",neutral
12834,speech,2008-04-21,Developing Sustainable Capital for Community I...,Governor Randall S. Kroszner,2789,<NA>,NaN,2008-04-30,-1,2.00,Good afternoon. I am pleased to address the in...,Community Reinvestment Fund's annual community...,neutral
13231,speech,2015-12-03,Financial Stability and Shadow Banks: What We ...,Vice Chairman Stanley Fischer,2608,<NA>,NaN,2017-06-14,1,NaN,"I have two goals this afternoon. First, I will...",Federal Reserve chief: Financial vulnerabiliti...,positive
12477,speech,2004-01-03,Fedspeak,Governor Ben S. Bernanke,4481,<NA>,NaN,2004-01-28,0,1.00,There was a time when central bankers did not ...,Central banks around the world have become not...,positive
13931,testimony,2006-06-08,Nomination as Vice Chairman to the Board,Governor Donald L. Kohn,376,<NA>,NaN,2006-06-29,1,5.25,"Chairman Shelby, Senator Sarbanes, and members...",I am honored to have been nominated by Preside...,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13229,speech,2015-12-02,The Economic Outlook and Monetary Policy,Janet Yellen,3644,<NA>,NaN,2017-06-14,1,NaN,Thank you to the Economic Club of Washington f...,U.S. economy has recovered substantially since...,positive
1880,presconf_script,2022-11-02,FOMC Press Conference Transcript,MICHELLE SMITH. Nick. NICK TIMIRAOS,276,1,3.75,2022-12-14,1,4.25,Nick Timiraos of the Wall Street Journal. Chai...,core PCE inflation on a 3- or 6-month annuali...,neutral
13380,speech,2019-01-10,Monetary Policy Outlook for 2019,Vice Chairman Richard H. Clarida,2457,<NA>,NaN,2019-01-30,0,2.25,Happy New Year. I am very glad to be speaking ...,Federal Reserve Chairman Jerome Powell: Initia...,neutral
166,meeting_script,2003-08-12,FOMC Meeting Transcript,,33571,0,1.00,2003-09-15,0,1.00,Without objection. The next item on the agenda...,Dino Kos: Fixed-income markets in the intermee...,neutral


In [11]:
submuestra_df.head()

,type,date,title,speaker,word_count,decision,rate,next_meeting,next_decision,next_rate,text,summary,predicted_roberta
714,presconf_script,2013-03-20,FOMC Press Conference Transcript,Ben Bernanke,391,0,0.0,2013-05-01,0,0.00,"Well, I still believe the following, which is ...","Bernanke: ""Monetary policy is a very blunt ins...",neutral
12834,speech,2008-04-21,Developing Sustainable Capital for Community I...,Governor Randall S. Kroszner,2789,<NA>,NaN,2008-04-30,-1,2.00,Good afternoon. I am pleased to address the in...,Community Reinvestment Fund's annual community...,neutral
13231,speech,2015-12-03,Financial Stability and Shadow Banks: What We ...,Vice Chairman Stanley Fischer,2608,<NA>,NaN,2017-06-14,1,NaN,"I have two goals this afternoon. First, I will...",Federal Reserve chief: Financial vulnerabiliti...,positive
12477,speech,2004-01-03,Fedspeak,Governor Ben S. Bernanke,4481,<NA>,NaN,2004-01-28,0,1.00,There was a time when central bankers did not ...,Central banks around the world have become not...,positive
13931,testimony,2006-06-08,Nomination as Vice Chairman to the Board,Governor Donald L. Kohn,376,<NA>,NaN,2006-06-29,1,5.25,"Chairman Shelby, Senator Sarbanes, and members...",I am honored to have been nominated by Preside...,positive


In [15]:
selected_columns_df = submuestra_df[['date', 'predicted_roberta', 'next_meeting','next_decision']]

In [16]:
# Define el diccionario de mapeo
mapping = {'negative': -1, 'positive': 1, 'neutral': 0}

# Reemplaza los valores en la columna 'predicted_roberta'
selected_columns_df['predicted_roberta'] = selected_columns_df['predicted_roberta'].replace(mapping)

In [17]:
selected_columns_df.head()

,date,predicted_roberta,next_meeting,next_decision
714,2013-03-20,0,2013-05-01,0
12834,2008-04-21,0,2008-04-30,-1
13231,2015-12-03,1,2017-06-14,1
12477,2004-01-03,1,2004-01-28,0
13931,2006-06-08,1,2006-06-29,1


Cargar datos numericos

In [12]:
# Load previously processed non-text data
# Load data
file = open('C:/Users/luisl/OneDrive/Escritorio/TFM_IA3/tfm_ia3_nodata/data/train_data/nontext_train_small.pickle', 'rb')
train_df = pickle.load(file)
file.close()

print(train_df.shape)
train_df

(403, 13)


,target,RateDiff,prev_decision,Employ_diff_year,Rsales_diff_year,Unemp_diff_year,Employ_diff_prev,GDP_diff_year,PCE_diff_prev,Unemp_diff_prev,Hsales_diff_year,GDP_diff_prev,Taylor
date,,,,,,,,,,,,,
1982-10-05,-1,-0.50,0,-2294.0,2.004317,-2.5,-180.0,-1.010745,0.460401,-0.3,42.307692,0.456177,11.061999
1982-11-16,-1,-0.50,-1,-2473.0,2.004317,-2.5,-276.0,-2.556042,0.370245,-0.3,34.831461,-0.382267,10.045779
1982-12-21,0,0.00,-1,-2385.0,2.004317,-2.5,-121.0,-2.556042,0.634292,-0.4,45.026178,-0.382267,10.105638
1983-01-14,0,0.00,0,-2124.0,2.004317,-2.3,-15.0,-2.556042,0.368789,0.0,14.004376,-0.382267,9.766001
1983-01-21,0,0.00,0,-2124.0,2.004317,-2.3,-15.0,-2.556042,0.368789,0.0,14.004376,-0.382267,9.766001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-14,0,0.00,1,3928.0,-1.938580,-0.1,281.0,1.717927,0.306901,-0.3,11.635220,0.556381,12.123858
2023-07-26,1,0.25,0,3663.0,-1.573436,0.0,105.0,1.717927,0.288262,0.1,21.314387,0.556381,12.011106
2023-09-20,0,0.00,1,3144.0,-0.840916,-0.1,165.0,2.382468,0.119978,-0.3,2.507837,0.511122,11.295164


In [13]:
train_df.reset_index(inplace=True)

In [14]:
train_df.columns

Index(['date', 'target', 'RateDiff', 'prev_decision', 'Employ_diff_year',
       'Rsales_diff_year', 'Unemp_diff_year', 'Employ_diff_prev',
       'GDP_diff_year', 'PCE_diff_prev', 'Unemp_diff_prev', 'Hsales_diff_year',
       'GDP_diff_prev', 'Taylor'],
      dtype='object')

unir los dos datasets, ponderando entre fechas el sentimientyo medio

In [59]:
df1= selected_columns_df
df2 = train_df

In [61]:
import pandas as pd
import numpy as np

# Crea una lista para almacenar los datos fusionados
merged_data = []

# Itera sobre cada fila de df1
for index, row in df1.iterrows():
    # Verifica si 'next_meeting' es un valor NaN o si no es del tipo datetime
    if pd.isnull(row['next_meeting']) or not isinstance(row['next_meeting'], pd.Timestamp):
        continue  # Salta esta fila si 'next_meeting' no es válido
    
    # Filtra df1 para las fechas en el rango definido por la fila actual de df1
    subset_df1 = df1[(df1['date'] >= row['date']) & (df1['date'] <= row['next_meeting'])]
    
    # Calcula el promedio de 'predicted_roberta' en el rango de fechas
    predicted_roberta_avg = subset_df1['predicted_roberta'].mean()
    
    # Añade los datos al resultado fusionado
    merged_data.append({
        'date': row['date'],
        'predicted_roberta': predicted_roberta_avg
    })

# Crea un DataFrame a partir de la lista de datos fusionados
merged_df = pd.DataFrame(merged_data)

# Fusiona 'merged_df' con 'df2' en la columna 'date'
merged_df = pd.merge(merged_df, df2, on='date', how='inner')

# Ordena el DataFrame por la columna 'date'
merged_df = merged_df.sort_values(by='date')




In [62]:

# Ordena el DataFrame por la columna 'date'
merged_df = merged_df.sort_values(by='date')

In [65]:
# Elimina las filas duplicadas del DataFrame
merged_df.drop_duplicates(inplace=True)

# Ordena el DataFrame por la columna 'date'
merged_df = merged_df.sort_values(by='date')


In [67]:
merged_df.drop(columns=['level_0', 'index'], inplace=True)

In [69]:
# Realiza la fusión de los DataFrames por la columna 'date'
merged_df = pd.merge(merged_df, df1[['date', 'next_meeting', 'next_decision']], on='date', how='inner')

# Ordena el DataFrame por la columna 'date'
merged_df = merged_df.sort_values(by='date')

In [71]:
# Elimina las filas duplicadas del DataFrame
merged_df.drop_duplicates(inplace=True)

df unificado

In [72]:
merged_df.tail()

,date,predicted_roberta,target,RateDiff,prev_decision,Employ_diff_year,Rsales_diff_year,Unemp_diff_year,Employ_diff_prev,GDP_diff_year,PCE_diff_prev,Unemp_diff_prev,Hsales_diff_year,GDP_diff_prev,Taylor,next_meeting,next_decision
2264,2023-05-03,0.105263,1,0.25,1,4048.0,-2.662041,0.1,217.0,1.717927,0.335926,0.1,-9.476662,0.556381,12.153794,2023-06-14,0
2291,2023-06-14,0.087912,0,0.00,1,3928.0,-1.938580,-0.1,281.0,1.717927,0.306901,-0.3,11.635220,0.556381,12.123858,2023-07-26,1
2339,2023-07-26,0.126437,1,0.25,0,3663.0,-1.573436,0.0,105.0,1.717927,0.288262,0.1,21.314387,0.556381,12.011106,2023-09-20,0
2372,2023-09-20,0.084211,0,0.00,1,3144.0,-0.840916,-0.1,165.0,2.382468,0.119978,-0.3,2.507837,0.511122,11.295164,2023-11-01,0
2411,2023-11-01,0.022727,0,0.00,0,3056.0,0.295139,-0.3,262.0,2.926887,0.329011,0.0,23.456790,1.193871,10.970821,2023-12-13,0
